<script async src="https://www.googletagmanager.com/gtag/js?id=UA-59152712-8"></script>
<script>
  window.dataLayer = window.dataLayer || [];
  function gtag(){dataLayer.push(arguments);}
  gtag('js', new Date());

  gtag('config', 'UA-59152712-8');
</script>

# Converting ADM Initial Data in the Spherical or Cartesian Basis to BSSN Initial Data in the Desired Curvilinear Basis
## Author: Zach Etienne

[comment]: <> (Abstract: TODO)

### This module is meant for use with any initial data that can be represented numerically in ADM form, either in the Spherical or Cartesian basis. I.e., the ADM variables are given $\left\{\gamma_{ij}, K_{ij}, \alpha, \beta^i\right\}$ *numerically* as functions of $(r,\theta,\phi)$ or $(x,y,z)$; e.g., through an initial data solver.

**Notebook Status:** <font color='orange'><b> Self-Validated </b></font>

**Validation Notes:** This tutorial notebook has been confirmed to be self-consistent with its corresponding NRPy+ module, as documented [below](#code_validation). **Additional validation tests may have been performed, but are as yet, undocumented. (TODO)**

### NRPy+ Source Code for this module: [BSSN/Tutorial-ADM_Initial_Data_Reader__BSSN_Converter.py](../edit/BSSN/Tutorial-ADM_Initial_Data_Reader__BSSN_Converter.py)



## Introduction:
Given the ADM variables:

$$\left\{\gamma_{ij}, K_{ij}, \alpha, \beta^i, B^i\right\}$$

in the Spherical or Cartesian basis, and as functions of $(r,\theta,\phi)$ or $(x,y,z)$, respectively, this module documents their conversion to the BSSN variables

$$\left\{\bar{\gamma}_{i j},\bar{A}_{i j},\phi, K, \bar{\Lambda}^{i}, \alpha, \beta^i, B^i\right\},$$ 

in the desired curvilinear basis (given by `reference_metric::CoordSystem`). Then it rescales the resulting BSSNCurvilinear variables (as defined in [the BSSN Curvilinear tutorial](Tutorial-BSSNCurvilinear.ipynb)) into the form needed for BSSNCurvilinear evolutions:

$$\left\{h_{i j},a_{i j},\phi, K, \lambda^{i}, \alpha, \mathcal{V}^i, \mathcal{B}^i\right\}.$$ 

We will use as our core example in this module UIUC initial data, which are ([as documented in their NRPy+ initial data module](Tutorial-ADM_Initial_Data-UIUC_BlackHole.ipynb)) given in terms of ADM variables in Spherical coordinates.

<a id='toc'></a>

# Table of Contents
$$\label{toc}$$ 

This notebook is organized as follows

1. [Step 1](#initializenrpy): Initialize core Python/NRPy+ modules
1. [Step 2](#read_convert_adm_to_cartesian_bssn): `initial_data_reader__convert_to_BSSN_from_ADM_sph_or_Cart()`: Read or compute ADM variables at all points on all grids, and convert them to BSSN curvilinear
1. [Step 3](#basis_xform): `initial_data_BSSN_basis_transform_Cartesian_to_rfm()`: Convert BSSN vectors/tensors from Cartesian to reference-metric basis
1. [Step 4](#lambda): `initial_data_lambdaU_grid_interior()`: Compute $\lambda^i$ from finite-difference derivatives of rescaled metric quantities
1. [Step 5](#nbd): `register_NRPy_basic_defines()`: Register `ID_data_struct` with `NRPy_basic_defines.h
1. [Step 6](#code_validation):Code Validation against  `BSSN.ADM_Initial_Data_Reader__BSSN_Converter` NRPy+ module
1. [Step 7](#latex_pdf_output): Output this notebook to $\LaTeX$-formatted PDF file

<a id='initializenrpy'></a>

# Step 1: Initialize core Python/NRPy+ modules \[Back to [top](#toc)\]
$$\label{initializenrpy}$$

In [1]:
# Step 1: Initialize core Python/NRPy+ modules and parameters
# Step 1.a: Add the NRPy+ base directory to the path
import os,sys,shutil            # Standard Python modules for multiplatform OS-level functions
nrpy_dir_path = os.path.join("..")
if nrpy_dir_path not in sys.path:
    sys.path.append(nrpy_dir_path)

# Step 1: Initialize core Python/NRPy+ modules
from outputC import outputC,lhrh,add_to_Cfunction_dict, Cfunction # NRPy+: Core C code output module
from outputC import outC_NRPy_basic_defines_h_dict
import NRPy_param_funcs as par    # NRPy+: Parameter interface
import finite_difference as fin   # NRPy+: Finite difference C code generation module
import grid as gri                # NRPy+: Functions having to do with numerical grids
import indexedexp as ixp          # NRPy+: Symbolic indexed expression (e.g., tensors, vectors, etc.) support
import reference_metric as rfm    # NRPy+: Reference metric support
import BSSN.BSSN_quantities as Bq # NRPy+: Computes useful BSSN quantities; e.g., gammabarUU & GammabarUDD needed below
import cmdline_helper as cmd      # NRPy+: Multi-platform Python command-line interface
from pickling import pickle_NRPy_env # NRPy+: Pickle/unpickle NRPy+ environment, for parallel codegen
import os, shutil, sys            # Standard Python modules for multiplatform OS-level functions

# Step 1.a: Create output directory for C codes generated by this tutorial:
Ccodesrootdir = os.path.join("ADM_initial_data_to_BSSN_converter")
# First remove C code output directory if it exists
# Courtesy https://stackoverflow.com/questions/303200/how-do-i-remove-delete-a-folder-that-is-not-empty
# !rm -r ScalarWaveCurvilinear_Playground_Ccodes
shutil.rmtree(Ccodesrootdir, ignore_errors=True)
# Then create a fresh directory
cmd.mkdir(Ccodesrootdir)

<a id='read_convert_adm_to_cartesian_bssn'></a>

# Step 2: `initial_data_reader_convert_to_BSSN_rfm_from_ADM_Sph_or_Cart()`: Read or compute ADM variables at all points on all grids, and convert them to BSSN curvilinear \[Back to [top](#toc)\]
$$\label{read_convert_adm_to_cartesian_bssn}$$

Initial data for constructing spacetimes are always provided in either the spherical or Cartesian bases   `initial_data_reader_convert_to_BSSN_rfm_from_ADM_Spherical()` and `initial_data_reader_convert_to_BSSN_rfm_from_ADM_Cartesian()` are the primary initial data read-in routines for BSSN initial data NRPy+ in a given reference metric. These routines do the following:

1. Call `ID_function()`: Read or compute ADM initial data at Cartesian point `xCart[3]`=$(x,y,z)$, in the spherical or Cartesian basis (`initial_data_reader__convert_to_BSSN_from_ADM_Spherical()` or `initial_data_reader__convert_to_BSSN_from_ADM_Cartesian()` respectively.
1. Call `ADM_SphorCart_to_Cart()`: Convert ADM variables from the spherical or Cartesian basis to the Cartesian basis.
1. Call `ADM_Cart_to_BSSN_Cart()`: Convert ADM variables in the Cartesian basis to BSSN variables in the Cartesian basis.
1. Call `BSSN_Cart_to_rescaled_BSSN_rfm()`: Convert all BSSN vectors/tensors *except* $\lambda^i$ in the Cartesian basis, to the basis specified by `reference_metric::CoordSystem`. Rescale BSSN quantities.
1. Call `initial_data_lambdaU_grid_interior()` to compute $\lambda^i$ in the `reference_metric::CoordSystem` basis, *in the grid interior only*.

**Important Note**: After `initial_data_reader__convert_to_BSSN_rfm_from_ADM_sph_or_Cart()` is called, inner/outer boundary conditions must be applied to $\lambda^i$ to ensure it is specified on the grid boundaries.

<a id='adm_sphorcart_to_cart'></a>

## Step 2.a: `ADM_SphorCart_to_Cart()`: Convert ADM variables from the spherical or Cartesian basis to the Cartesian basis \[Back to [top](#toc)\]
$$\label{adm_sphorcart_to_cart}$$

In [2]:
def Cfunction_ADM_SphorCart_to_Cart(input_Coord="Spherical", include_T4UU=False):
    includes = []

    desired_rfm_basis = par.parval_from_str("reference_metric::CoordSystem")

    desc = "Convert ADM variables from the spherical or Cartesian basis to the Cartesian basis"
    c_type = "static void"
    name = "ADM_SphorCart_to_Cart"
    params = """paramstruct *restrict params, const REAL xCart[3], const initial_data_struct *restrict initial_data,
                                  ADM_Cart_basis_struct *restrict ADM_Cart_basis"""

    body = r"""
  // Unpack initial_data for scalar alpha
  const REAL alpha = initial_data->alpha;

  // Unpack initial_data for ADM vectors/tensors
"""
    for i in ["betaSphorCartU", "BSphorCartU"]:
        for j in range(3):
            varname = i + str(j)
            body += "  const REAL " + varname + " = initial_data->" + varname + ";\n"
        body += "\n"
    for i in ["gammaSphorCartDD", "KSphorCartDD"]:
        for j in range(3):
            for k in range(j, 3):
                varname = i + str(j) + str(k)
                body += "  const REAL " + varname + " = initial_data->" + varname + ";\n"
        body += "\n"
    # Read stress-energy tensor in spherical or Cartesian basis if desired.
    if include_T4UU:
        for mu in range(4):
            for nu in range(mu, 4):
                varname = "T4SphorCartUU" + str(mu) + str(nu)
                body += "  const REAL " + varname + " = initial_data->" + varname + ";\n"
        body += "\n"

    body += r"""
  // Perform the basis transform on ADM vectors/tensors from """+input_Coord+""" to Cartesian:
  {
    // Set destination point
    const REAL Cartx = xCart[0];
    const REAL Carty = xCart[1];
    const REAL Cartz = xCart[2];

    // Set destination xx[3]
"""
    # Set reference_metric to the input_Coord
    par.set_parval_from_str("reference_metric::CoordSystem", input_Coord)
    rfm.reference_metric()

    body += outputC(rfm.Cart_to_xx[:3], ["const REAL xx0", "const REAL xx1", "const REAL xx2"],
                    filename="returnstring",
                    params="outCverbose=False,includebraces=False,preindent=2,CSE_varprefix=tmp_xx")

    # Define the input variables:
    gammaSphorCartDD = ixp.declarerank2("gammaSphorCartDD", "sym01")
    KSphorCartDD     = ixp.declarerank2("KSphorCartDD", "sym01")
    betaSphorCartU = ixp.declarerank1("betaSphorCartU")
    BSphorCartU    = ixp.declarerank1("BSphorCartU")
    T4SphorCartUU = ixp.declarerank2("T4SphorCartUU", "sym01", DIM=4)

    # Compute Jacobian to convert to Cartesian coordinates
    Jac_dUCart_dDrfmUD, Jac_dUrfm_dDCartUD = rfm.compute_Jacobian_and_inverseJacobian_tofrom_Cartesian()

    gammaCartDD = rfm.basis_transform_tensorDD_from_rfmbasis_to_Cartesian(Jac_dUrfm_dDCartUD, gammaSphorCartDD)
    KCartDD = rfm.basis_transform_tensorDD_from_rfmbasis_to_Cartesian(Jac_dUrfm_dDCartUD, KSphorCartDD)
    betaCartU = rfm.basis_transform_vectorU_from_rfmbasis_to_Cartesian(Jac_dUCart_dDrfmUD, betaSphorCartU)
    BCartU = rfm.basis_transform_vectorU_from_rfmbasis_to_Cartesian(Jac_dUCart_dDrfmUD, BSphorCartU)
    T4CartUU = ixp.zerorank2(DIM=4)
    if include_T4UU:
        T4CartUU = rfm.basis_transform_4tensorUU_from_CartorSph_to_rfm(T4SphorCartUU, CoordType_in=input_Coord)

    list_of_output_exprs    = []
    list_of_output_varnames = []
    for i in range(3):
        list_of_output_exprs += [betaCartU[i]]
        list_of_output_varnames += ["ADM_Cart_basis->betaU" + str(i)]
        list_of_output_exprs += [BCartU[i]]
        list_of_output_varnames += ["ADM_Cart_basis->BU" + str(i)]
        for j in range(i, 3):
            list_of_output_exprs += [gammaCartDD[i][j]]
            list_of_output_varnames += ["ADM_Cart_basis->gammaDD" + str(i) + str(j)]
            list_of_output_exprs += [KCartDD[i][j]]
            list_of_output_varnames += ["ADM_Cart_basis->KDD" + str(i) + str(j)]
    if include_T4UU:
        for mu in range(4):
            for nu in range(mu, 4):
                list_of_output_exprs += [T4CartUU[mu][nu]]
                list_of_output_varnames += ["ADM_Cart_basis->T4UU" + str(mu) + str(nu)]

    # Sort the outputs before calling outputC()
    # https://stackoverflow.com/questions/9764298/is-it-possible-to-sort-two-listswhich-reference-each-other-in-the-exact-same-w
    list_of_output_varnames, list_of_output_exprs = (list(t) for t in zip(*sorted(zip(list_of_output_varnames, list_of_output_exprs))))

    body += outputC(list_of_output_exprs, list_of_output_varnames,
                    filename="returnstring", params="outCverbose=False,includebraces=False,preindent=2")
    body += r"""
  }
"""
    # Restore reference metric globals to coordsystem on grid.
    par.set_parval_from_str("reference_metric::CoordSystem", desired_rfm_basis)
    rfm.reference_metric()

    _func_prototype, func = Cfunction(
        includes=includes,
        desc=desc,
        c_type=c_type, name=name, params=params,
        body=body,
        enableCparameters=False)
    return func

<a id='adm_cart_to_bssn_cart'></a>

## Step 2.b: `ADM_Cart_to_BSSN_Cart()`: Convert ADM variables in the Cartesian basis to BSSN variables in the Cartesian basis \[Back to [top](#toc)\]
$$\label{adm_cart_to_bssn_cart}$$

In [3]:
def Cfunction_ADM_Cart_to_BSSN_Cart(include_T4UU=False):
    includes = []

    desired_rfm_basis = par.parval_from_str("reference_metric::CoordSystem")

    desc = "Convert ADM variables in the Cartesian basis to BSSN variables in the Cartesian basis"
    c_type = "static void"
    name = "ADM_Cart_to_BSSN_Cart"
    params = """paramstruct *restrict params, const REAL xCart[3], const ADM_Cart_basis_struct *restrict ADM_Cart_basis,
                                  BSSN_Cart_basis_struct *restrict BSSN_Cart_basis"""

    # Extract desired rfm basis from reference_metric::CoordSystem
    desired_rfm_basis = par.parval_from_str("reference_metric::CoordSystem")

    # Set CoordSystem to Cartesian
    par.set_parval_from_str("reference_metric::CoordSystem", "Cartesian")
    rfm.reference_metric()

    gammaCartDD = ixp.declarerank2("ADM_Cart_basis->gammaDD", "sym01")
    KCartDD     = ixp.declarerank2("ADM_Cart_basis->KDD", "sym01")

    import BSSN.BSSN_in_terms_of_ADM as BitoA
    BitoA.trK_AbarDD_aDD(gammaCartDD, KCartDD)
    BitoA.gammabarDD_hDD(gammaCartDD)
    BitoA.cf_from_gammaDD(gammaCartDD)

    body = r"""
  // *In the Cartesian basis*, convert ADM quantities gammaDD & KDD
  //   into BSSN gammabarDD, AbarDD, cf, and trK.
  BSSN_Cart_basis->alpha = ADM_Cart_basis->alpha;
  BSSN_Cart_basis->betaU0 = ADM_Cart_basis->betaU0;
  BSSN_Cart_basis->betaU1 = ADM_Cart_basis->betaU1;
  BSSN_Cart_basis->betaU2 = ADM_Cart_basis->betaU2;
  BSSN_Cart_basis->BU0 = ADM_Cart_basis->BU0;
  BSSN_Cart_basis->BU1 = ADM_Cart_basis->BU1;
  BSSN_Cart_basis->BU2 = ADM_Cart_basis->BU2;
"""
    list_of_output_exprs    = [BitoA.cf, BitoA.trK]
    list_of_output_varnames = ["BSSN_Cart_basis->cf", "BSSN_Cart_basis->trK"]
    for i in range(3):
        for j in range(i, 3):
            list_of_output_exprs += [BitoA.gammabarDD[i][j]]
            list_of_output_varnames += ["BSSN_Cart_basis->gammabarDD" + str(i) + str(j)]
            list_of_output_exprs += [BitoA.AbarDD[i][j]]
            list_of_output_varnames += ["BSSN_Cart_basis->AbarDD" + str(i) + str(j)]
    if include_T4UU:
        T4CartUU = ixp.declarerank2("ADM_Cart_basis->T4UU", "sym01", DIM=4)
        for mu in range(4):
            for nu in range(mu, 4):
                list_of_output_exprs += [T4CartUU[mu][nu]]
                list_of_output_varnames += ["BSSN_Cart_basis->T4UU" + str(mu) + str(nu)]

    # Sort the outputs before calling outputC()
    # https://stackoverflow.com/questions/9764298/is-it-possible-to-sort-two-listswhich-reference-each-other-in-the-exact-same-w
    list_of_output_varnames, list_of_output_exprs = (list(t) for t in zip(*sorted(zip(list_of_output_varnames, list_of_output_exprs))))
    body += outputC(list_of_output_exprs, list_of_output_varnames,
                    filename="returnstring", params="outCverbose=False,includebraces=False,preindent=1")

    # Restore reference metric globals to desired reference metric.
    par.set_parval_from_str("reference_metric::CoordSystem", desired_rfm_basis)
    rfm.reference_metric()

    _func_prototype, func = Cfunction(
        includes=includes,
        desc=desc,
        c_type=c_type, name=name, params=params,
        body=body,
        enableCparameters=False)
    return func

<a id='bssn_cart_to_rescaled_bssn_rfm'></a>

## Step 2.c: `BSSN_Cart_to_rescaled_BSSN_rfm()`: Convert Cartesian-basis BSSN vectors/tensors *except* $\lambda^i$, to the basis specified by `reference_metric::CoordSystem`, then rescale these BSSN quantities. \[Back to [top](#toc)\]
$$\label{bssn_cart_to_rescaled_bssn_rfm}$$

By the time this function is called, all BSSN tensors and vectors are in the Cartesian coordinate basis $x^i_{\rm Cart} = (x,y,z)$, but we need them in the curvilinear coordinate basis $x^i_{\rm rfm}=$`(xx0,xx1,xx2)` set by the `"reference_metric::CoordSystem"` variable. Empirically speaking, it is far easier to write `(x(xx0,xx1,xx2),y(xx0,xx1,xx2),z(xx0,xx1,xx2))` than the inverse, so we will compute the Jacobian matrix

$$
{\rm Jac\_dUCart\_dDrfmUD[i][j]} = \frac{\partial x^i_{\rm Cart}}{\partial x^j_{\rm rfm}},
$$

via exact differentiation (courtesy SymPy), and the inverse Jacobian
$$
{\rm Jac\_dUrfm\_dDCartUD[i][j]} = \frac{\partial x^i_{\rm rfm}}{\partial x^j_{\rm Cart}},
$$

using NRPy+'s `generic_matrix_inverter3x3()` function. In terms of these, the transformation of BSSN tensors from Spherical to `"reference_metric::CoordSystem"` coordinates may be written:

\begin{align}
\beta^i_{\rm rfm} &= \frac{\partial x^i_{\rm rfm}}{\partial x^\ell_{\rm Cart}} \beta^\ell_{\rm Cart}\\
B^i_{\rm rfm} &= \frac{\partial x^i_{\rm rfm}}{\partial x^\ell_{\rm Cart}} B^\ell_{\rm Cart}\\
\bar{\gamma}^{\rm rfm}_{ij} &= 
\frac{\partial x^\ell_{\rm Cart}}{\partial x^i_{\rm rfm}}
\frac{\partial x^m_{\rm Cart}}{\partial x^j_{\rm rfm}} \bar{\gamma}^{\rm Cart}_{\ell m}\\
\bar{A}^{\rm rfm}_{ij} &= 
\frac{\partial x^\ell_{\rm Cart}}{\partial x^i_{\rm rfm}}
\frac{\partial x^m_{\rm Cart}}{\partial x^j_{\rm rfm}} \bar{A}^{\rm Cart}_{\ell m}
\end{align}

The above basis transforms are included in functions `basis_transform_tensorDD_from_Cartesian_to_rfmbasis()` and `basis_transform_vectorU_from_Cartesian_to_rfmbasis()` in `reference_metric.py`, and we use them below.

After the basis transform has been performed, we perform tensor rescalings to compute the evolved variables $h_{ij}$, $a_{ij}$, $\text{vet}^i$, and $\text{bet}^i$:

$\bar{\gamma}_{ij}$ is rescaled $h_{ij}$ according to the prescription described in the [the covariant BSSN formulation tutorial](Tutorial-BSSN_formulation.ipynb) (also [Ruchlin *et al.*](https://arxiv.org/pdf/1712.07658.pdf)):

$$
h_{ij} = (\bar{\gamma}_{ij} - \hat{\gamma}_{ij})/\text{ReDD[i][j]}.
$$

Further $\bar{A}_{ij}$, $\beta^i$, $B^i$ are rescaled to $a_{ij}$, $\text{vet}^i$, and $\text{bet}^i$ respectively via the standard formulas (found in [the covariant BSSN formulation tutorial](Tutorial-BSSN_formulation.ipynb); also [Ruchlin *et al.*](https://arxiv.org/pdf/1712.07658.pdf)):

\begin{align}
a_{ij} &= \bar{A}_{ij}/\text{ReDD[i][j]} \\
\text{vet}^i &= \beta^i/\text{ReU[i]} \\
\text{bet}^i &= B^i/\text{ReU[i]}.
\end{align}

In [4]:
def Cfunction_BSSN_Cart_to_rescaled_BSSN_rfm(include_T4UU=False):
    includes = []

    desc = r"""Convert Cartesian-basis BSSN vectors/tensors *except* lambda^i,
to the basis specified by `reference_metric::CoordSystem`, then rescale these BSSN quantities"""
    c_type = "static void"
    name = "BSSN_Cart_to_rescaled_BSSN_rfm"
    params = """paramstruct *restrict params, const REAL xCart[3],
                                           const BSSN_Cart_basis_struct *restrict BSSN_Cart_basis,
                                           rescaled_BSSN_rfm_basis_struct *restrict rescaled_BSSN_rfm_basis"""

    body = "  const REAL Cartx=xCart[0], Carty=xCart[1], Cartz=xCart[2];\n"
    # We're in the rfm coordinate basis now.
    body += outputC(rfm.Cart_to_xx[:3], ["const REAL xx0", "const REAL xx1", "const REAL xx2"],
                   filename="returnstring",
                   params="outCverbose=False,includebraces=False,preindent=1,CSE_varprefix=tmp_xx")

    # Define the input variables:
    gammabarCartDD = ixp.declarerank2("BSSN_Cart_basis->gammabarDD", "sym01")
    AbarCartDD     = ixp.declarerank2("BSSN_Cart_basis->AbarDD", "sym01")
    betaCartU = ixp.declarerank1("BSSN_Cart_basis->betaU")
    BCartU    = ixp.declarerank1("BSSN_Cart_basis->BU")

    # Compute Jacobian to convert to Cartesian coordinates
    Jac_dUCart_dDrfmUD, Jac_dUrfm_dDCartUD = rfm.compute_Jacobian_and_inverseJacobian_tofrom_Cartesian()

    gammabarDD = rfm.basis_transform_tensorDD_from_Cartesian_to_rfmbasis(Jac_dUCart_dDrfmUD, gammabarCartDD)
    AbarDD = rfm.basis_transform_tensorDD_from_Cartesian_to_rfmbasis(Jac_dUCart_dDrfmUD, AbarCartDD)
    betaU = rfm.basis_transform_vectorU_from_Cartesian_to_rfmbasis(Jac_dUrfm_dDCartUD, betaCartU)
    BU = rfm.basis_transform_vectorU_from_Cartesian_to_rfmbasis(Jac_dUrfm_dDCartUD, BCartU)

    # Next rescale:
    vetU = ixp.zerorank1()
    betU = ixp.zerorank1()
    hDD  = ixp.zerorank2()
    aDD  = ixp.zerorank2()
    for i in range(3):
        vetU[i] = betaU[i] / rfm.ReU[i]
        betU[i] =    BU[i] / rfm.ReU[i]
        for j in range(3):
            hDD[i][j] = (gammabarDD[i][j] - rfm.ghatDD[i][j]) / rfm.ReDD[i][j]
            aDD[i][j] = AbarDD[i][j] / rfm.ReDD[i][j]

    rescaled_T4UU = ixp.zerorank2(DIM=4)
    if include_T4UU:
        T4CartUU = ixp.declarerank2("BSSN_Cart_basis->T4UU", "sym01", DIM=4)
        T4UU = rfm.basis_transform_4tensorUU_from_CartorSph_to_rfm(T4CartUU, CoordType_in="Cartesian")
        rescaled_T4UU = ixp.zerorank2(DIM=4)
        for mu in range(4):
            for nu in range(mu, 4):
                rescaled_T4UU[mu][nu] = T4UU[mu][nu]
        for mu in range(1, 4):
            for nu in range(mu, 4):
                rescaled_T4UU[mu][nu] = T4UU[mu][nu] * rfm.ReDD[(mu-1)][(nu-1)]

    list_of_output_exprs    = []
    list_of_output_varnames = []
    for i in range(3):
        list_of_output_exprs += [vetU[i]]
        list_of_output_varnames += ["rescaled_BSSN_rfm_basis->vetU" + str(i)]
        list_of_output_exprs += [betU[i]]
        list_of_output_varnames += ["rescaled_BSSN_rfm_basis->betU" + str(i)]
        for j in range(i, 3):
            list_of_output_exprs += [hDD[i][j]]
            list_of_output_varnames += ["rescaled_BSSN_rfm_basis->hDD" + str(i) + str(j)]
            list_of_output_exprs += [aDD[i][j]]
            list_of_output_varnames += ["rescaled_BSSN_rfm_basis->aDD" + str(i) + str(j)]
    if include_T4UU:
        for mu in range(4):
            for nu in range(mu, 4):
                list_of_output_exprs += [rescaled_T4UU[mu][nu]]
                list_of_output_varnames += ["rescaled_BSSN_rfm_basis->T4UU" + str(mu) + str(nu)]

    # Sort the outputs before calling outputC()
    # https://stackoverflow.com/questions/9764298/is-it-possible-to-sort-two-listswhich-reference-each-other-in-the-exact-same-w
    list_of_output_varnames, list_of_output_exprs = (list(t) for t in zip(*sorted(zip(list_of_output_varnames, list_of_output_exprs))))

    body += outputC(list_of_output_exprs, list_of_output_varnames,
                    filename="returnstring", params="outCverbose=False,includebraces=False,preindent=1")

    _func_prototype, func = Cfunction(
        includes=includes,
        desc=desc,
        c_type=c_type, name=name, params=params,
        body=body,
        enableCparameters=False)
    return func

<a id='lambda'></a>

## Step 2.d: `initial_data_lambdaU_grid_interior()`: Compute $\lambda^i$ from finite-difference derivatives of rescaled metric quantities \[Back to [top](#toc)\]
$$\label{lambda}$$

We compute $\bar{\Lambda}^i$ (Eqs. 4 and 5 of [Baumgarte *et al.*](https://arxiv.org/pdf/1211.6632.pdf)), from finite-difference derivatives of rescaled metric quantities $h_{ij}$:

$$
\bar{\Lambda}^i = \bar{\gamma}^{jk}\left(\bar{\Gamma}^i_{jk} - \hat{\Gamma}^i_{jk}\right).
$$

The [reference_metric.py](../edit/reference_metric.py) module provides us with analytic expressions for $\hat{\Gamma}^i_{jk}$, so here we need only compute finite-difference expressions for $\bar{\Gamma}^i_{jk}$, based on the values for $h_{ij}$ provided in the initial data. Once $\bar{\Lambda}^i$ has been computed, we apply the usual rescaling procedure:
$$
\lambda^i = \bar{\Lambda}^i/\text{ReU[i]},
$$
and then output the result to a C file using the NRPy+ finite-difference C output routine.

In [5]:
# initial_data_lambdaU_grid_interior() computes lambdaU from
#  finite-difference derivatives of rescaled metric quantities
def Cfunction_initial_data_lambdaU_grid_interior():
    includes = []
    c_type = "static void"

    output_Coord = par.parval_from_str("reference_metric::CoordSystem")
    desc = "Compute lambdaU in " + output_Coord + " coordinates"
    name = "initial_data_lambdaU_grid_interior"
    params = """const paramstruct *restrict params, REAL *restrict xx[3], REAL *restrict in_gfs"""
    # Step 7: Compute $\bar{\Lambda}^i$ from finite-difference derivatives of rescaled metric quantities

    # We will need all BSSN gridfunctions to be defined, as well as
    #     expressions for gammabarDD_dD in terms of exact derivatives of
    #     the rescaling matrix and finite-difference derivatives of
    #     hDD's. This functionality is provided by BSSN.BSSN_unrescaled_and_barred_vars,
    #     which we call here to overwrite above definitions of gammabarDD,gammabarUU, etc.
    Bq.gammabar__inverse_and_derivs()  # Provides gammabarUU and GammabarUDD
    gammabarUU    = Bq.gammabarUU
    GammabarUDD   = Bq.GammabarUDD

    # Next evaluate \bar{\Lambda}^i, based on GammabarUDD above and GammahatUDD
    #       (from the reference metric):
    LambdabarU = ixp.zerorank1()
    for i in range(3):
        for j in range(3):
            for k in range(3):
                LambdabarU[i] += gammabarUU[j][k] * (GammabarUDD[i][j][k] - rfm.GammahatUDD[i][j][k])

    # Finally apply rescaling:
    # lambda^i = Lambdabar^i/\text{ReU[i]}
    lambdaU = ixp.zerorank1()
    for i in range(3):
        lambdaU[i] = LambdabarU[i] / rfm.ReU[i]

    lambdaU_expressions = [lhrh(lhs=gri.gfaccess("in_gfs", "lambdaU0"), rhs=lambdaU[0]),
                           lhrh(lhs=gri.gfaccess("in_gfs", "lambdaU1"), rhs=lambdaU[1]),
                           lhrh(lhs=gri.gfaccess("in_gfs", "lambdaU2"), rhs=lambdaU[2])]
    body = fin.FD_outputC("returnstring", lambdaU_expressions,
                           params="outCverbose=False,includebraces=False,preindent=0")
    _func_prototype, func = Cfunction(
        includes=includes,
        desc=desc,
        c_type=c_type, name=name, params=params,
        body=body,
        loopopts="InteriorPoints,Read_xxs",
        enableCparameters=True)
    return func

In [6]:
def add_to_Cfunction_dict_initial_data_reader__convert_to_BSSN_rfm_from_ADM_Sph_or_Cart(input_Coord="Spherical",
                                                                                        include_T4UU=False):
    def T4UU_prettyprint():
        return r"""
  REAL T4UU00,T4UU01,T4UU02,T4UU03;
  REAL        T4UU11,T4UU12,T4UU13;
  REAL               T4UU22,T4UU23;
  REAL                      T4UU33;
"""
    prefunc = """
// ADM variables in the Cartesian basis:
typedef struct __ADM_Cart_basis_struct__ {
  REAL alpha, betaU0,betaU1,betaU2, BU0,BU1,BU2;
  REAL gammaDD00,gammaDD01,gammaDD02,gammaDD11,gammaDD12,gammaDD22;
  REAL KDD00,KDD01,KDD02,KDD11,KDD12,KDD22;
"""
    if include_T4UU:
        prefunc += T4UU_prettyprint()
    prefunc += "} ADM_Cart_basis_struct;\n"
    ##############
    prefunc += """
// BSSN variables in the Cartesian basis:
typedef struct __BSSN_Cart_basis_struct__ {
  REAL alpha, betaU0,betaU1,betaU2, BU0,BU1,BU2;
  REAL cf, trK;
  REAL gammabarDD00,gammabarDD01,gammabarDD02,gammabarDD11,gammabarDD12,gammabarDD22;
  REAL AbarDD00,AbarDD01,AbarDD02,AbarDD11,AbarDD12,AbarDD22;
"""
    if include_T4UU:
        prefunc += T4UU_prettyprint()
    prefunc += "} BSSN_Cart_basis_struct;\n"
    ##############
    prefunc += """
// Rescaled BSSN variables in the rfm basis:
typedef struct __rescaled_BSSN_rfm_basis_struct__ {
  REAL alpha, vetU0,vetU1,vetU2, betU0,betU1,betU2;
  REAL cf, trK;
  REAL hDD00,hDD01,hDD02,hDD11,hDD12,hDD22;
  REAL aDD00,aDD01,aDD02,aDD11,aDD12,aDD22;
"""
    if include_T4UU:
        prefunc += T4UU_prettyprint()
    prefunc += "} rescaled_BSSN_rfm_basis_struct;\n"
    ##############
    ##############
    prefunc += Cfunction_ADM_SphorCart_to_Cart(input_Coord=input_Coord, include_T4UU=include_T4UU)
    prefunc += Cfunction_ADM_Cart_to_BSSN_Cart(                         include_T4UU=include_T4UU)
    prefunc += Cfunction_BSSN_Cart_to_rescaled_BSSN_rfm(include_T4UU=include_T4UU)
    prefunc += Cfunction_initial_data_lambdaU_grid_interior()
    includes = ["NRPy_basic_defines.h", "NRPy_function_prototypes.h"]

    output_Coord = par.parval_from_str("reference_metric::CoordSystem")
    desc = "Read in ADM initial data in the " + input_Coord + " basis, and convert to BSSN data in " + output_Coord + " coordinates"
    c_type = "void"
    name = "initial_data_reader__convert_to_BSSN_from_ADM_" + input_Coord
    params = """griddata_struct *restrict griddata, ID_persist_struct *restrict ID_persist,
                                                             void ID_function(const paramstruct *params, const REAL xCart[3],
                                                                              const ID_persist_struct *restrict ID_persist,
                                                                              initial_data_struct *restrict initial_data)"""

    body = r"""
  const int Nxx_plus_2NGHOSTS0 = griddata->params.Nxx_plus_2NGHOSTS0;
  const int Nxx_plus_2NGHOSTS1 = griddata->params.Nxx_plus_2NGHOSTS1;
  const int Nxx_plus_2NGHOSTS2 = griddata->params.Nxx_plus_2NGHOSTS2;

  LOOP_OMP("omp parallel for", i0,0,Nxx_plus_2NGHOSTS0, i1,0,Nxx_plus_2NGHOSTS1, i2,0,Nxx_plus_2NGHOSTS2) {
    // xCart is the global Cartesian coordinate, which accounts for any grid offsets from the origin.
    REAL xCart[3];  xx_to_Cart(&griddata->params, griddata->xx, i0,i1,i2, xCart);

    // Read or compute initial data at destination point xCart
    initial_data_struct initial_data;
    ID_function(&griddata->params, xCart, ID_persist, &initial_data);

    ADM_Cart_basis_struct ADM_Cart_basis;
    ADM_SphorCart_to_Cart(&griddata->params, xCart, &initial_data, &ADM_Cart_basis);

    BSSN_Cart_basis_struct BSSN_Cart_basis;
    ADM_Cart_to_BSSN_Cart(&griddata->params, xCart, &ADM_Cart_basis, &BSSN_Cart_basis);

    rescaled_BSSN_rfm_basis_struct rescaled_BSSN_rfm_basis;
    BSSN_Cart_to_rescaled_BSSN_rfm(&griddata->params, xCart, &BSSN_Cart_basis, &rescaled_BSSN_rfm_basis);

    const int idx3 = IDX3S(i0,i1,i2);

    // Output data to gridfunctions
"""
    gf_list = ["alpha", "trK", "cf"]
    for i in range(3):
        gf_list += ["vetU"+str(i), "betU"+str(i)]
        for j in range(i, 3):
            gf_list += ["hDD"+str(i)+str(j), "aDD"+str(i)+str(j)]
    for gf in sorted(gf_list):
        body += "    griddata->gridfuncs.y_n_gfs[IDX4ptS("+gf.upper()+"GF, idx3)] = rescaled_BSSN_rfm_basis."+gf+";\n"
    if include_T4UU:
        for mu in range(4):
            for nu in range(mu, 4):
                gf = "T4UU" + str(mu) + str(nu)
                body += "    griddata->gridfuncs.auxevol_gfs[IDX4ptS("+gf.upper()+"GF, idx3)] = rescaled_BSSN_rfm_basis."+gf+";\n"
    body += """
  } // END LOOP over all gridpoints on given grid

  initial_data_lambdaU_grid_interior(&griddata->params, griddata->xx, griddata->gridfuncs.y_n_gfs);
"""

    add_to_Cfunction_dict(
        includes=includes,
        prefunc=prefunc,
        desc=desc,
        c_type=c_type, name=name, params=params,
        body=body,
        enableCparameters=False)
    return pickle_NRPy_env()

<a id='exact'></a>

# Step 5: Exact (closed-form expression) ADM initial data driver function \[Back to [top](#toc)\]
$$\label{exact}$$

This function converts closed-form SymPy expressions for $\alpha$, $\beta^i$, $B^i$, $\gamma_{ij}$ and $K_{ij}$ in the spherical or Cartesian basis at a given point $(x,y,z)$ to optimized C code. It is fully compatible with other functions here.

In [7]:
def add_to_Cfunction_dict_exact_ADM_ID_function(IDtype, IDCoordSystem, alpha, betaU, BU, gammaDD, KDD):
    includes = ["NRPy_basic_defines.h"]
    desc = IDtype + " initial data"
    c_type = "void"
    name = IDtype
    params = "const paramstruct *params, const REAL xCart[3], const ID_persist_struct *restrict ID_persist, initial_data_struct *restrict initial_data"
    desired_rfm_coord = par.parval_from_str("reference_metric::CoordSystem")
    par.set_parval_from_str("reference_metric::CoordSystem", IDCoordSystem)
    rfm.reference_metric()
    body = ""
    if IDCoordSystem == "Spherical":
        body += r"""  const REAL Cartx=xCart[0], Carty=xCart[1], Cartz=xCart[2];
  REAL xx0,xx1,xx2 __attribute__((unused));  // xx2 might be unused in the case of axisymmetric initial data.
  {
""" + outputC(rfm.Cart_to_xx[:3], ["xx0", "xx1", "xx2"], filename="returnstring",
              params="outCverbose=False,includebraces=False,preindent=2") + """
  }
  const REAL r  = xx0; // Some ID only specify r,th,ph.
  const REAL th = xx1;
  const REAL ph = xx2;
"""
    elif IDCoordSystem == "Cartesian":
        body += r"""  const REAL xx0=xCart[0], xx1=xCart[1], xx2=xCart[2];
"""
    else:
        print("add_to_Cfunction_dict_exact_ADM_ID_function() Error: IDCoordSystem == " + IDCoordSystem + " unsupported")
        sys.exit(1)
    list_of_output_exprs = [alpha]
    list_of_output_varnames = ["initial_data->alpha"]
    for i in range(3):
        list_of_output_exprs += [betaU[i]]
        list_of_output_varnames += ["initial_data->betaSphorCartU" + str(i)]
        list_of_output_exprs += [BU[i]]
        list_of_output_varnames += ["initial_data->BSphorCartU" + str(i)]
        for j in range(i, 3):
            list_of_output_exprs += [gammaDD[i][j]]
            list_of_output_varnames += ["initial_data->gammaSphorCartDD" + str(i) + str(j)]
            list_of_output_exprs += [KDD[i][j]]
            list_of_output_varnames += ["initial_data->KSphorCartDD" + str(i) + str(j)]
    # Sort the outputs before calling outputC()
    # https://stackoverflow.com/questions/9764298/is-it-possible-to-sort-two-listswhich-reference-each-other-in-the-exact-same-w
    list_of_output_varnames, list_of_output_exprs = (list(t) for t in zip(*sorted(zip(list_of_output_varnames, list_of_output_exprs))))

    body += outputC(list_of_output_exprs, list_of_output_varnames,
                    filename="returnstring", params="outCverbose=False,includebraces=False,preindent=1")

    # Restore CoordSystem:
    par.set_parval_from_str("reference_metric::CoordSystem", desired_rfm_coord)
    rfm.reference_metric()
    add_to_Cfunction_dict(
        includes=includes,
        desc=desc, c_type=c_type, name=name, params=params,
        body=body,
        enableCparameters=True)
    return pickle_NRPy_env()

<a id='nbd'></a>

# Step 5: `register_NRPy_basic_defines()`: Register `ID_data_struct` with `NRPy_basic_defines.h` \[Back to [top](#toc)\]
$$\label{nbd}$$

Other than its core use as a means to store ADM input quantities, `ID_data_struct` is designed to be extensible. For example, it may be used to store e.g., pseudospectral coefficients for TwoPunctures, initial data gridfunctions from NRPyElliptic, pointers to TOV 1D data from the TOV solver, etc.

In [8]:
# Other than its core use as a means to store ADM input quantities,
# `initial_data_struct` is designed to be extensible. For example, it may be
# used to store e.g., pseudospectral coefficients for TwoPunctures,
# initial data gridfunctions from NRPyElliptic, pointers to TOV 1D data
# from the TOV solver, etc.
def register_NRPy_basic_defines(ID_persist_struct_contents_str="", include_T4UU=False):
    Nbd = r"""typedef struct __initial_data_struct__ {
  REAL alpha;

  REAL betaSphorCartU0, betaSphorCartU1, betaSphorCartU2;
  REAL BSphorCartU0, BSphorCartU1, BSphorCartU2;

  REAL gammaSphorCartDD00, gammaSphorCartDD01, gammaSphorCartDD02;
  REAL gammaSphorCartDD11, gammaSphorCartDD12, gammaSphorCartDD22;

  REAL KSphorCartDD00, KSphorCartDD01, KSphorCartDD02;
  REAL KSphorCartDD11, KSphorCartDD12, KSphorCartDD22;
"""
    if include_T4UU:
        Nbd += """
  REAL T4SphorCartUU00,T4SphorCartUU01,T4SphorCartUU02,T4SphorCartUU03;
  REAL                 T4SphorCartUU11,T4SphorCartUU12,T4SphorCartUU13;
  REAL                                 T4SphorCartUU22,T4SphorCartUU23;
  REAL                                                 T4SphorCartUU33;
"""
    Nbd += """
} initial_data_struct;
"""
    Nbd += "typedef struct __ID_persist_struct__ {\n"
    Nbd += ID_persist_struct_contents_str + "\n"
    Nbd += "} ID_persist_struct;\n"
    outC_NRPy_basic_defines_h_dict["BSSN_initial_data"] = Nbd

<a id='code_validation'></a>

# Step 6: Code Validation against  `BSSN.ADM_Initial_Data_Reader__BSSN_Converter` NRPy+ module \[Back to [top](#toc)\]
$$\label{code_validation}$$

Here, as a code validation check, we verify agreement in the C codes for converting "numerical" UIUCBlackHole initial data (in Spherical coordinates/basis) to BSSN Curvilinear data in Cylindrical coordinates/basis between
1. this tutorial and 
2. the NRPy+ [BSSN.ADM_Initial_Data_Reader__BSSN_Converter](../edit/BSSN/ADM_Initial_Data_Reader__BSSN_Converter.py) module.

By default, we analyze these expressions in Cylindrical coordinates, though other coordinate systems may be chosen.

In [9]:
# import BSSN.ADM_Initial_Data_Reader__BSSN_Converter as AID

# funclist = [(add_to_Cfunction_dict_initial_data_reader__convert_to_BSSN_from_ADM_sph_or_Cart, AID.add_to_Cfunction_dict_initial_data_reader__convert_to_BSSN_from_ADM_sph_or_Cart),
#             (add_to_Cfunction_dict_initial_data_BSSN_basis_transform_Cartesian_to_rfm_and_rescale, AID.add_to_Cfunction_dict_initial_data_BSSN_basis_transform_Cartesian_to_rfm_and_rescale),
#             (add_to_Cfunction_dict_initial_data_lambdaU_grid_interior, AID.add_to_Cfunction_dict_initial_data_lambdaU_grid_interior),
#             (add_to_Cfunction_dict_exact_ADM_ID_function, AID.add_to_Cfunction_dict_exact_ADM_ID_function),
#             (register_NRPy_basic_defines, AID.register_NRPy_basic_defines)
#            ]

# import inspect
# for func in funclist:
#     # https://stackoverflow.com/questions/20059011/check-if-two-python-functions-are-equal
#     if inspect.getsource(func[0]) != inspect.getsource(func[1]):
#         with open(func[0].__name__ + "_Jupyter_notebook_version.c", "w") as file:
#             file.write(inspect.getsource(func[0]))
#         with open(func[1].__name__ + "_Python_module_version.c", "w") as file:
#             file.write(inspect.getsource(func[1]))
#         print("ERROR: function " + func[0].__name__ + " is not the same as the Ccodegen_library version!")
#         print(" For more info, try this:")
#         print("diff " + func[0].__name__ + "_Jupyter_notebook_version.c" + " " + func[1].__name__ + "_Python_module_version.c")
#         sys.exit(1)

# print("PASS! ALL FUNCTIONS ARE IDENTICAL")

In [10]:
##########################
# Polytropic EOS example #
##########################
import TOV.Polytropic_EOSs as ppeos
EOS_type = "SimplePolytrope"

if EOS_type == "SimplePolytrope":
    # Set neos = 1 (single polytrope)
    neos = 1

    # Set rho_poly_tab (not needed for a single polytrope)
    rho_poly_tab = []

    # Set Gamma_poly_tab
    Gamma_poly_tab = [2.0]

    # Set K_poly_tab0
    K_poly_tab0 = 1. # ZACH NOTES: CHANGED FROM 100.

    # Set the eos quantities
    eos = ppeos.set_up_EOS_parameters__complete_set_of_input_variables(neos,rho_poly_tab,Gamma_poly_tab,K_poly_tab0)
    rho_baryon_central = 0.129285
elif EOS_type == "PiecewisePolytrope":
    eos = ppeos.set_up_EOS_parameters__Read_et_al_input_variables(EOS_name)
    rho_baryon_central=2.0
else:
    print("""Error: unknown EOS_type. Valid types are 'SimplePolytrope' and 'PiecewisePolytrope' """)
    sys.exit(1)

import TOV.TOV_Solver as TOV
M_TOV, R_Schw_TOV, R_iso_TOV = TOV.TOV_Solver(eos,
                                              outfile=os.path.join(Ccodesrootdir, "outputTOVpolytrope.txt"),
                                              rho_baryon_central=rho_baryon_central,
                                              return_M_RSchw_and_Riso = True,
                                              verbose = True)

# domain_size sets the default value for:
#   * Spherical's params.RMAX
#   * SinhSpherical*'s params.AMAX
#   * Cartesians*'s -params.{x,y,z}min & .{x,y,z}max
#   * Cylindrical's -params.ZMIN & .{Z,RHO}MAX
#   * SinhCylindrical's params.AMPL{RHO,Z}
#   * *SymTP's params.AMAX
domain_size = 2.0 * R_iso_TOV

1256 1256 1256 1256 1256 1256
Just generated a TOV star with
* M        = 1.405030336771405e-01 ,
* R_Schw   = 9.566044579232513e-01 ,
* R_iso    = 8.100085557410308e-01 ,
* M/R_Schw = 1.468768334847266e-01 



In [11]:
def add_to_Cfunction_dict_TOV_read_data_file_set_ID_persist():
    includes = ["NRPy_basic_defines.h"]
    desc = "Returns the number of lines in a TOV data file."
    c_type = "void"
    name = "TOV_read_data_file_set_ID_persist"
    params = "const char *input_filename, ID_persist_struct *ID_persist"
    body = r"""
  char filename[100];
  snprintf(filename, 100, input_filename);
  FILE *in1Dpolytrope = fopen(filename, "r");
  if(in1Dpolytrope == NULL) {
    fprintf(stderr,"ERROR: could not open file %s\n",filename);
    exit(1);
  }

  int numlines_in_file = 0;
  {
    char * line = NULL;

    size_t len = 0;
    ssize_t read;
    while ((read = getline(&line, &len, in1Dpolytrope)) != -1) {
      numlines_in_file++;
    }
    rewind(in1Dpolytrope);

    free(line);
  }

  // Now that numlines_in_file is set, we can now allocate memory for all arrays.
  {
    ID_persist->r_Schw_arr     = (REAL *restrict)malloc(sizeof(REAL)*numlines_in_file);
    ID_persist->rho_arr        = (REAL *restrict)malloc(sizeof(REAL)*numlines_in_file);
    ID_persist->rho_baryon_arr = (REAL *restrict)malloc(sizeof(REAL)*numlines_in_file);
    ID_persist->P_arr          = (REAL *restrict)malloc(sizeof(REAL)*numlines_in_file);
    ID_persist->M_arr          = (REAL *restrict)malloc(sizeof(REAL)*numlines_in_file);
    ID_persist->expnu_arr      = (REAL *restrict)malloc(sizeof(REAL)*numlines_in_file);
    ID_persist->exp4phi_arr    = (REAL *restrict)malloc(sizeof(REAL)*numlines_in_file);
    ID_persist->rbar_arr       = (REAL *restrict)malloc(sizeof(REAL)*numlines_in_file);
  }

  {
    char * line = NULL;

    size_t len = 0;
    ssize_t read;

    int which_line = 0;
    while ((read = getline(&line, &len, in1Dpolytrope)) != -1) {
      // Define the line delimiters (i.e., the stuff that goes between the data on a given
      //     line of data.  Here, we define both spaces " " and tabs "\t" as data delimiters.
      const char delimiters[] = " \t";

      // Now we define "token", a pointer to the first column of data
      char *token;

      // Each successive time we call strtok(NULL,blah), we read in a new column of data from
      //     the originally defined character array, as pointed to by token.

      token=strtok(line, delimiters); if(token==NULL) { fprintf(stderr, "Error reading %s\n", filename); exit(1); }
      ID_persist->r_Schw_arr[which_line]     = strtod(token, NULL); token = strtok( NULL, delimiters );
      ID_persist->rho_arr[which_line]        = strtod(token, NULL); token = strtok( NULL, delimiters );
      ID_persist->rho_baryon_arr[which_line] = strtod(token, NULL); token = strtok( NULL, delimiters );
      ID_persist->P_arr[which_line]          = strtod(token, NULL); token = strtok( NULL, delimiters );
      ID_persist->M_arr[which_line]          = strtod(token, NULL); token = strtok( NULL, delimiters );
      ID_persist->expnu_arr[which_line]      = strtod(token, NULL); token = strtok( NULL, delimiters );
      ID_persist->exp4phi_arr[which_line]    = strtod(token, NULL); token = strtok( NULL, delimiters );
      ID_persist->rbar_arr[which_line]       = strtod(token, NULL);

      which_line++;
    }
    free(line);
  }
"""
    add_to_Cfunction_dict(
        includes=includes,
        desc=desc,
        c_type=c_type, name=name, params=params,
        body=body,
        enableCparameters=False)

In [12]:
def add_to_Cfunction_dict_TOV_interpolate_1D():
    includes=["NRPy_basic_defines.h"]
    prefunc = r"""
// Find interpolation index using Bisection root-finding algorithm:
static inline int bisection_idx_finder(const REAL rrbar, const int numlines_in_file, const REAL *restrict rbar_arr) {
  int x1 = 0;
  int x2 = numlines_in_file-1;
  REAL y1 = rrbar-rbar_arr[x1];
  REAL y2 = rrbar-rbar_arr[x2];
  if(y1*y2 >= 0) {
    fprintf(stderr,"INTERPOLATION BRACKETING ERROR %e | %e %e\n",rrbar,y1,y2);
    exit(1);
  }
  for(int i=0;i<numlines_in_file;i++) {
    int x_midpoint = (x1+x2)/2;
    REAL y_midpoint = rrbar-rbar_arr[x_midpoint];
    if(y_midpoint*y1 < 0) {
      x2 = x_midpoint;
      y2 = y_midpoint;
    } else {
      x1 = x_midpoint;
      y1 = y_midpoint;
    }
    if( abs(x2-x1) == 1 ) {
      // If rbar_arr[x1] is closer to rrbar than rbar_arr[x2] then return x1:
      if(fabs(rrbar-rbar_arr[x1]) < fabs(rrbar-rbar_arr[x2])) return x1;
      // Otherwiser return x2:
      return x2;
    }
  }
  fprintf(stderr,"INTERPOLATION BRACKETING ERROR: DID NOT CONVERGE.\n");
  exit(1);
}
"""
    desc = """Read a TOV solution from data file and perform
1D interpolation of the solution to a desired radius.

Author: Zachariah B. Etienne
        zachetie **at** gmail **dot* com
"""
    c_type="void"
    name="TOV_interpolate_1D"
    params="""REAL rrbar,const ID_persist_struct *ID_persist,
                        REAL *restrict rho,REAL *restrict rho_baryon,REAL *restrict P,REAL *restrict M,REAL *restrict expnu,REAL *restrict exp4phi"""
    body = r"""
  // First unpack ID_persist struct (this should be pretty quick relative to the rest of the routine):
  const REAL Rbar               = ID_persist->Rbar;
  const int Rbar_idx            = ID_persist->Rbar_idx;
  const int interp_stencil_size = ID_persist->interp_stencil_size;
  const int numlines_in_file    = ID_persist->numlines_in_file;
  const REAL *restrict rbar_arr       = ID_persist->rbar_arr;
  const REAL *restrict r_Schw_arr     = ID_persist->r_Schw_arr;
  const REAL *restrict rho_arr        = ID_persist->rho_arr;
  const REAL *restrict rho_baryon_arr = ID_persist->rho_baryon_arr;
  const REAL *restrict P_arr          = ID_persist->P_arr;
  const REAL *restrict M_arr          = ID_persist->M_arr;
  const REAL *restrict expnu_arr      = ID_persist->expnu_arr;
  const REAL *restrict exp4phi_arr    = ID_persist->exp4phi_arr;

  // For this case, we know that for all functions, f(r) = f(-r)
  if(rrbar < 0) rrbar = -rrbar;

  // First find the central interpolation stencil index:
  int idx = bisection_idx_finder(rrbar,numlines_in_file,rbar_arr);


  int idxmin = MAX(0,idx-interp_stencil_size/2-1);

  // -= Do not allow the interpolation stencil to cross the star's surface =-
  // max index is when idxmin + (interp_stencil_size-1) = Rbar_idx
  //  -> idxmin at most can be Rbar_idx - interp_stencil_size + 1
  if(rrbar < Rbar) {
    idxmin = MIN(idxmin,Rbar_idx - interp_stencil_size + 1);
  } else {
    idxmin = MAX(idxmin,Rbar_idx+1);
    idxmin = MIN(idxmin,numlines_in_file - interp_stencil_size + 1);
  }
  // Now perform the Lagrange polynomial interpolation:

  // First set the interpolation coefficients:
  REAL rbar_sample[interp_stencil_size];
  for(int i=idxmin;i<idxmin+interp_stencil_size;i++) {
    rbar_sample[i-idxmin] = rbar_arr[i];
  }
  REAL l_i_of_r[interp_stencil_size];
  for(int i=0;i<interp_stencil_size;i++) {
    REAL numer = 1.0;
    REAL denom = 1.0;
    for(int j=0;j<i;j++) {
      numer *= rrbar - rbar_sample[j];
      denom *= rbar_sample[i] - rbar_sample[j];
    }
    for(int j=i+1;j<interp_stencil_size;j++) {
      numer *= rrbar - rbar_sample[j];
      denom *= rbar_sample[i] - rbar_sample[j];
    }
    l_i_of_r[i] = numer/denom;
  }

  // Then perform the interpolation:
  *rho = 0.0;
  *rho_baryon = 0.0;
  *P = 0.0;
  *M = 0.0;
  *expnu = 0.0;
  *exp4phi = 0.0;

  REAL r_Schw = 0.0;
  for(int i=idxmin;i<idxmin+interp_stencil_size;i++) {
    r_Schw      += l_i_of_r[i-idxmin] * r_Schw_arr[i];
    *rho        += l_i_of_r[i-idxmin] * rho_arr[i];
    *rho_baryon += l_i_of_r[i-idxmin] * rho_baryon_arr[i];
    *P          += l_i_of_r[i-idxmin] * P_arr[i];
    *M          += l_i_of_r[i-idxmin] * M_arr[i];
    *expnu      += l_i_of_r[i-idxmin] * expnu_arr[i];
    *exp4phi    += l_i_of_r[i-idxmin] * exp4phi_arr[i];
  }

  if(rrbar > Rbar) {
    *rho        = 0;
    *rho_baryon = 0;
    *P          = 0;
    *M          = M_arr[Rbar_idx+1];
    *expnu      = 1. - 2.*(*M) / r_Schw;
    *exp4phi    = pow(r_Schw / rrbar,2.0);
  }
"""
    add_to_Cfunction_dict(
        includes=includes,
        prefunc=prefunc,
        desc=desc,
        c_type=c_type, name=name, params=params,
        body=body,
        enableCparameters=False)

In [13]:
import sympy as sp

def add_to_Cfunction_dict_TOV_ID_function():
    includes = ["NRPy_basic_defines.h", "NRPy_function_prototypes.h"]
    desc = """This function sets TOV initial data at a single point xCart[3] = (x,y,z),
outputting to initial_data_struct *restrict initial_data"""
    c_type = "void"
    name = "TOV_ID_function"
    params = "const paramstruct *params, const REAL xCart[3], const ID_persist_struct *restrict ID_persist, initial_data_struct *restrict initial_data"
    body  = """  // First set r(=rbar), theta, phi in terms of xCart[3]:
  const REAL Cartx=xCart[0], Carty=xCart[1], Cartz=xCart[2];"""
    desired_rfm_coord = par.parval_from_str("reference_metric::CoordSystem")
    # "Spherical" is the native basis for TOV initial data.
    par.set_parval_from_str("reference_metric::CoordSystem", "Spherical")
    rfm.reference_metric()
    body += r"""
  REAL rbar, theta, phi;
  {
""" + outputC(rfm.Cart_to_xx[:3], ["rbar", "theta", "phi"], filename="returnstring",
              params="outCverbose=False,includebraces=False,preindent=2")
    body += r"""
  }

  // Next set gamma_{ij} in spherical basis
  REAL rho,rho_baryon,P,M,expnu,exp4phi;
  TOV_interpolate_1D(rbar,ID_persist, &rho,&rho_baryon,&P,&M,&expnu,&exp4phi);
"""
    # in TOV ID, betaU=BU=KDD=0
    betaU = ixp.zerorank1()
    BU    = ixp.zerorank1()
    KDD   = ixp.zerorank2()

    rbar,theta,rho,P,expnu,exp4phi = sp.symbols('rbar theta rho P expnu exp4phi', real=True)
    IDalpha = sp.sqrt(expnu)
    gammaSphDD = ixp.zerorank2(DIM=3)
    gammaSphDD[0][0] = exp4phi
    gammaSphDD[1][1] = exp4phi*rbar**2
    gammaSphDD[2][2] = exp4phi*rbar**2*sp.sin(theta)**2

    T4SphUU = ixp.zerorank2(DIM=4)
    T4SphUU[0][0] = rho/expnu
    T4SphUU[1][1] = P/exp4phi
    T4SphUU[2][2] = P/(exp4phi*rbar**2)
    T4SphUU[3][3] = P/(exp4phi*rbar**2*sp.sin(theta)**2)

    list_of_output_exprs = [IDalpha]
    list_of_output_varnames = ["initial_data->alpha"]
    for i in range(3):
        list_of_output_exprs += [betaU[i]]
        list_of_output_varnames += ["initial_data->betaSphorCartU" + str(i)]
        list_of_output_exprs += [BU[i]]
        list_of_output_varnames += ["initial_data->BSphorCartU" + str(i)]
        for j in range(i, 3):
            list_of_output_exprs += [gammaSphDD[i][j]]
            list_of_output_varnames += ["initial_data->gammaSphorCartDD" + str(i) + str(j)]
            list_of_output_exprs += [KDD[i][j]]
            list_of_output_varnames += ["initial_data->KSphorCartDD" + str(i) + str(j)]
    for mu in range(4):
        for nu in range(mu, 4):
            list_of_output_exprs += [T4SphUU[mu][nu]]
            list_of_output_varnames += ["initial_data->T4SphorCartUU" + str(mu) + str(nu)]

    # Sort the outputs before calling outputC()
    # https://stackoverflow.com/questions/9764298/is-it-possible-to-sort-two-listswhich-reference-each-other-in-the-exact-same-w
    list_of_output_varnames, list_of_output_exprs = (list(t) for t in zip(*sorted(zip(list_of_output_varnames, list_of_output_exprs))))

    body += outputC(list_of_output_exprs, list_of_output_varnames,
                    filename="returnstring", params="outCverbose=False,includebraces=False,preindent=1")

    # Restore CoordSystem:
    par.set_parval_from_str("reference_metric::CoordSystem", desired_rfm_coord)
    rfm.reference_metric()

    add_to_Cfunction_dict(
        includes=includes,
        desc=desc,
        c_type=c_type, name=name, params=params,
        body=body,
        enableCparameters=False)

In [14]:
add_to_Cfunction_dict_TOV_read_data_file_set_ID_persist()
add_to_Cfunction_dict_TOV_interpolate_1D()
add_to_Cfunction_dict_TOV_ID_function()

In [15]:
def add_to_Cfunction_dict_main__Initial_Data_Playground():
    includes = ["NRPy_basic_defines.h", "NRPy_function_prototypes.h", "time.h"]
    desc = """// main() function:
// Step 0: Read command-line input, set up grid structure, allocate memory for gridfunctions, set up coordinates
// Step 1: Set up initial data to an exact solution
// Step 2: Output data on xy plane to file.
// Step 3: Free all allocated memory
"""
    c_type = "int"
    name = "main"
    params = "int argc, const char *argv[]"
    body = r"""  griddata_struct griddata;
  set_Cparameters_to_default(&griddata.params);

  // Step 0.a: Set free parameters, overwriting Cparameters defaults
  //          by hand or with command-line input, as desired.
#include "free_parameters.h"

  // Step 0.b: Read command-line input, error out if nonconformant
  if((argc != 4) || atoi(argv[1]) < NGHOSTS || atoi(argv[2]) < NGHOSTS || atoi(argv[3]) < 2 /* FIXME; allow for axisymmetric sims */) {
    fprintf(stderr,"Error: Expected three command-line arguments: ./BrillLindquist_Playground Nx0 Nx1 Nx2,\n");
    fprintf(stderr,"where Nx[0,1,2] is the number of grid points in the 0, 1, and 2 directions.\n");
    fprintf(stderr,"Nx[] MUST BE larger than NGHOSTS (= %d)\n",NGHOSTS);
    exit(1);
  }
  // Step 0.c: Check grid structure, first in space...
  const int Nxx[3] = { atoi(argv[1]), atoi(argv[2]), atoi(argv[3]) };
  if(Nxx[0]%2 != 0 || Nxx[1]%2 != 0 || Nxx[2]%2 != 0) {
    fprintf(stderr,"Error: Cannot guarantee a proper cell-centered grid if number of grid cells not set to even number.\n");
    fprintf(stderr,"       For example, in case of angular directions, proper symmetry zones will not exist.\n");
    exit(1);
  }

  // Step 0.d: Uniform coordinate grids are stored to *xx[3]
  // Step 0.d.i: Set bcstruct
  {
    // Step 0.f: Call set_Nxx_dxx_invdx_params__and__xx(), which sets
    //          params Nxx,Nxx_plus_2NGHOSTS,dxx,invdx, and xx[] for the
    //          chosen (non-Eigen) CoordSystem.
    int EigenCoord = 0;
    set_Nxx_dxx_invdx_params__and__xx(EigenCoord, Nxx, &griddata.params, griddata.xx);
  }

  // Step 0.j: Allocate memory for y_n_gfs gridfunctions
  const int Nxx_plus_2NGHOSTS0 = griddata.params.Nxx_plus_2NGHOSTS0;
  const int Nxx_plus_2NGHOSTS1 = griddata.params.Nxx_plus_2NGHOSTS1;
  const int Nxx_plus_2NGHOSTS2 = griddata.params.Nxx_plus_2NGHOSTS2;
  const int grid_size = Nxx_plus_2NGHOSTS0*Nxx_plus_2NGHOSTS1*Nxx_plus_2NGHOSTS2;
  REAL *restrict y_n_gfs = (REAL *restrict)malloc(sizeof(REAL)*grid_size*NUM_EVOL_GFS);
  REAL *restrict auxevol_gfs = (REAL *restrict)malloc(sizeof(REAL)*grid_size*NUM_AUXEVOL_GFS);

  // Step 0.l: Set up initial data to an exact solution
    // Count the number of lines in the data file:
    int numlines_in_file = count_num_lines_in_file(in1Dpolytrope);

  REAL *restrict auxevol_gfs = (REAL *restrict)malloc(sizeof(REAL)*grid_size*NUM_AUXEVOL_GFS);
  REAL *restrict aux_gfs = (REAL *restrict)malloc(sizeof(REAL)*grid_size*NUM_AUX_GFS);

  // To simplify the expressions somewhat, we compute & store the Ricci tensor separately
  //    from the BSSN constraints.
  Ricci_eval(&griddata.params, griddata.xx, y_n_gfs, auxevol_gfs);
  BSSN_constraints(&griddata.params, griddata.xx, y_n_gfs, auxevol_gfs, aux_gfs);

  xy_plane_diagnostics(&griddata, y_n_gfs, aux_gfs);

  // Step 4: Free all allocated memory
  free(y_n_gfs);
  free(auxevol_gfs);
  free(aux_gfs);
  for(int i=0;i<3;i++) free(griddata.xx[i]);

  return 0;
"""
    add_to_Cfunction_dict(
        includes=includes,
        desc=desc,
        c_type=c_type, name=name, params=params,
        body=body,
        rel_path_to_Cparams=os.path.join("."), enableCparameters=False)

In [16]:
include_T4UU=True

add_to_Cfunction_dict_initial_data_reader__convert_to_BSSN_rfm_from_ADM_Sph_or_Cart(input_Coord="Spherical",
                                                                                    include_T4UU=include_T4UU)

ID_persist_struct_contents_str="""
  REAL Rbar;
  int Rbar_idx;
  int interp_stencil_size;
  int numlines_in_file;
  REAL *restrict r_Schw_arr;
  REAL *restrict rho_arr;
  REAL *restrict rho_baryon_arr;
  REAL *restrict P_arr;
  REAL *restrict M_arr;
  REAL *restrict expnu_arr;
  REAL *restrict exp4phi_arr;
  REAL *restrict rbar_arr;
"""
register_NRPy_basic_defines(include_T4UU=include_T4UU,
                            ID_persist_struct_contents_str=ID_persist_struct_contents_str)

if include_T4UU:
    _T4UU = ixp.register_gridfunctions_for_single_rank2("AUXEVOL", "T4UU", "sym01", DIM=4)

import BSSN.UIUCBlackHole as UIB
UIB.UIUCBlackHole(ComputeADMGlobalsOnly=True,include_NRPy_basic_defines_and_pickle=False)
add_to_Cfunction_dict_exact_ADM_ID_function("UIUCBlackHole", "Spherical",
                                            UIB.alphaSph, UIB.betaSphU, UIB.BSphU, UIB.gammaSphDD, UIB.KSphDD)

import outputC as outC
import finite_difference as fin
fin.register_C_functions_and_NRPy_basic_defines(NGHOSTS_account_for_onezone_upwind=True, enable_SIMD=False)

outC.outC_NRPy_basic_defines_h_dict["MoL"] = """
typedef struct __MoL_gridfunctions_struct__ {
  REAL *restrict y_n_gfs;
  REAL *restrict auxevol_gfs;
} MoL_gridfunctions_struct;
"""
par.register_NRPy_basic_defines()  # add `paramstruct params` to griddata struct.
list_of_extras_in_griddata_struct = ["MoL_gridfunctions_struct gridfuncs;"]
gri.register_C_functions_and_NRPy_basic_defines(list_of_extras_in_griddata_struct=list_of_extras_in_griddata_struct)  # #define IDX3S(), etc.

outC.outputC_register_C_functions_and_NRPy_basic_defines()
outC.NRPy_param_funcs_register_C_functions_and_NRPy_basic_defines(os.path.join(Ccodesrootdir))

# Call this last: Set up NRPy_basic_defines.h and NRPy_function_prototypes.h.
outC.construct_NRPy_basic_defines_h(Ccodesrootdir, enable_SIMD=False)
outC.construct_NRPy_function_prototypes_h(Ccodesrootdir)

outC.construct_Makefile_from_outC_function_dict(Ccodesrootdir, "", uses_free_parameters_h=False,
                                                compiler_opt_option="fastdebug", addl_CFLAGS=None,
                                                mkdir_Ccodesrootdir=True, use_make=True, CC="gcc",
                                                create_lib=True,  include_dirs=None)
# Now all the C codes generated in this notebook may be found in ADM_initial_data_to_BSSN_converter/
#    Go there and type `make` to see if it all compiles.

<a id='latex_pdf_output'></a>

# Step 7: Output this notebook to $\LaTeX$-formatted PDF file \[Back to [top](#toc)\]
$$\label{latex_pdf_output}$$

Once the following code finishes running, the generated PDF may be found at the following location within the directory you have the NRPy+ tutorial saved:
[Tutorial-ADM_Initial_Data_Reader__BSSN_Converter.pdf](Tutorial-ADM_Initial_Data_Reader__BSSN_Converter.pdf)

In [17]:
import cmdline_helper as cmd    # NRPy+: Multi-platform Python command-line interface
cmd.output_Jupyter_notebook_to_LaTeXed_PDF("Tutorial-ADM_Initial_Data_Reader__BSSN_Converter")

Created Tutorial-ADM_Initial_Data_Reader__BSSN_Converter.tex, and compiled
    LaTeX file to PDF file Tutorial-
    ADM_Initial_Data_Reader__BSSN_Converter.pdf
